# Sepsis-3 evaluation in the MIMIC-III database

This notebook goes over the evaluation of the new Sepsis-3 guidelines in the MIMIC database. The goals of this analysis include:

1. Evaluating the Sepsis-3 guidelines in MIMIC using the same methodology as in the research paper
2. Evaluating the Sepsis-3 guidelines against ANGUS criteria
3. Assessing if there are interesting subgroup(s) which are missed by the criteria

In [ ]:
from __future__ import print_function

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sepsis_utils import sepsis_utils as su
from sepsis_utils import roc_utils as ru

# used to calculate AUROC
from sklearn import metrics

# default colours for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']
%matplotlib inline

In [ ]:
# load data
df = pd.read_csv('sepsis3-df.csv',sep=',')
df_mdl = pd.read_csv('sepsis3-design-matrix.csv',sep=',')

# define outcome
target_header = "angus"
y = df[target_header].values == 1

# define the covariates to be added in the MFP model (used for table of AUROCs)
preds_header = ['sirs','sepsis3','mlods']

# Study questions

1. How well do the guidelines detect sepsis (Angus criteria) in the antibiotics/culture subset?
2. How well do the guidelines predict mortality (in-hospital) in the antibiotics/culture subset?
3. What factors would improve the sensitivity of the guidelines?
4. What factors would improve the specificity of the guidelines?

## Angus criteria evaluation

In [ ]:
yhat = df.sepsis3.values
print('\n SEPSIS-3 guidelines for Angus criteria sepsis \n')
print('Accuracy = {}'.format(metrics.accuracy_score(y, yhat)))
su.print_cm(y, yhat, header1='ang',header2='sep3') # print confusion matrix

Predictions using various levels of confounder adjustment are calculated in the subfunctions `calc_predictions`:

* `model=None` - the severity scores on their own
* `model='baseline'` - the severity scores in a vanilla regression
* `model='mfp'` -the severity scores in a fractional polynomial regression (calls an R script)

For Angus criteria we do not adjust for other factors when presenting the AUROCs.

In [ ]:
preds = su.calc_predictions(df, preds_header, target_header, model=None)

In [ ]:
# reproduce the AUC table
su.print_auc_table(preds, y, preds_header)

su.print_auc_table_to_file(preds, y, preds_header=preds_header,
                           filename='auc-table.csv')

In [ ]:
# define "predictions" according to the SEPSIS-3 guidelines:
#  suspicion of infection, qSOFA >= 2, and SOFA >= 2
yhat_all = [df.sepsis3.values,
            df.sirs.values >= 2,
            df.mlods.values >= 2]
yhat_names = ['seps3','SIRS', 'mLODS']

# define "targets", angus critera
y_all = [y for x in yhat_names]

stats_all = su.print_op_stats(yhat_all, y_all,
               yhat_names=yhat_names,
               header=target_header)

# Appendix

In the main body of the paper, we only evaluate operating point statistics for Angus criteria. Below, we reproduce ROC curves for the readers' interest.

In [ ]:
markers = {'qsofa': 'o:', 'sofa': '^-', 'sirs': 's-', 'mlods': 'd-'}
colors = {'qsofa': col[0], 'sofa': col[1], 'sirs': col[2], 'mlods': col[3]}
pred_pretty_text = {'qsofa': 'qSOFA', 'sofa': 'SOFA', 'sirs': 'SIRS', 'mlods': 'mLODS'}

# plot the data
plt.figure(figsize=[9,9])

# calculate and plot ROC for each score
for pred in ['sirs','qsofa','sofa','mlods']:
    if pred=='sepsis3':
        continue
        
    fpr, tpr, thr = metrics.roc_curve(y, df[pred].values)
    auc, auc_ci = ru.calc_auc(df[pred].values, y, with_ci=True, alpha=0.05)
    plt.plot(fpr, tpr, markers[pred],
             color=colors[pred], linewidth=2, markersize=10,
             label=pred_pretty_text[pred] + ', ' +
             '{:0.2f} [{:0.2f}, {:0.2f}]'.format(auc, auc_ci[0], auc_ci[1]))

plt.legend(loc="lower right")

plt.plot([0,1], [0,1], '--',
         color=[0,0,0], linewidth=2)
# reformat the plot
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
#plt.title('ROC against ' + target_header,fontsize=14)
plt.show()

## Histograms comparing qSOFA

In [ ]:
# histogram of the qSOFA values in septic/non-septic population
qsofa_alive = df.qsofa.values[~y]
qsofa_dead = df.qsofa.values[y]

xi = [-0.5,0.5,1.5,2.5,3.5]

prevalence = np.mean(y)

# plot the data
plt.figure(figsize=[9,9])
n0, bins0, patches0 = plt.hist(qsofa_alive, bins=xi, normed=True, color=col[0], alpha=0.5,
         label='qSOFA - ' + target_header + '=0')
n1, bins1, patches1 = plt.hist(qsofa_dead, bins=xi, normed=True, color=col[1], alpha=0.5,
         label='qSOFA - ' + target_header + '=1')

plt.legend(loc="upper right")

# reformat the plot
plt.xlim([-0.5,4.5])
#plt.ylim([-0.05, 1.05])
plt.xlabel('qSOFA',fontsize=14)
plt.ylabel('Proportion - normalized within groups',fontsize=14)

plt.figure(figsize=[9,9])

N = len(y)
plt.bar(bins0[0:-1], 100.0*n0/N*qsofa_alive.shape[0], width=1, color=col[0], alpha=0.5,
         label='qSOFA - ' + target_header + '=0')
plt.bar(bins1[0:-1], 100.0*n1/N*qsofa_dead.shape[0], width=1, color=col[1], alpha=0.5,
         label='qSOFA - ' + target_header + '=1')
plt.legend()
plt.ylabel('Percent of TOTAL patients')
plt.show()